In [ ]:
# 输入完整数据集，划分训练集验证集测试集的目标真实结果和按照给定缺失率缺失化后的结果。
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
SEED = int(time.time())

def generate_missing_data(dataset, missing_rate):
    # 对数据集主体部分进行缺失化处理，不包括最后一列（标签列）
    dataset_without_labels = dataset.iloc[:, :-1]  # 不包括标签列
    missing_dataset = dataset_without_labels.copy()
    total_values = missing_dataset.shape[0] * missing_dataset.shape[1]
    num_missing_values = int(total_values * missing_rate)
    indices = [(i, j) for i in range(missing_dataset.shape[0]) for j in range(missing_dataset.shape[1])]
    preserved_indices = {(i, random.randint(0, missing_dataset.shape[1] - 1)) for i in range(missing_dataset.shape[0])}
    indices = [idx for idx in indices if idx not in preserved_indices]
    random.shuffle(indices)
    for idx in indices[:num_missing_values]:
        missing_dataset.iloc[idx] = np.nan

    # 重新添加标签列
    missing_dataset = pd.concat([missing_dataset, dataset.iloc[:, -1]], axis=1)

    return missing_dataset

def mask_test_labels(dataset):
    # 将测试集的标签列全部替换为NaN
    dataset_with_masked_labels = dataset.copy()
    dataset_with_masked_labels.iloc[:, -1] = np.nan
    return dataset_with_masked_labels

# missing_rate = 0.3  # 设置缺失率
# MISSING_RATE = missing_rate*100
DATANAME = 'autompg'
# ID_NUM = 1
for ID_NUM in [1,2,3,4,5]:
    for missing_rate in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7]:
        MISSING_RATE = int(missing_rate*100)

        # 读取数据集
        dataset = pd.read_csv(f'datasets/{DATANAME}/{DATANAME}.csv',header=None)
        # 分割数据集为训练集、验证集、测试集;70%,10%,20%
        train_val, test_set = train_test_split(dataset, test_size=0.2, random_state=SEED+int(ID_NUM+MISSING_RATE))  # 测试集 20%
        train_set, val_set = train_test_split(train_val, test_size=0.125, random_state=SEED+int(ID_NUM+MISSING_RATE))  # 验证集 10% of 80% = 8% of total
        # 生成缺失数据
        missing_data_train = generate_missing_data(train_set, missing_rate)
        missing_data_val = generate_missing_data(val_set, missing_rate)
        missing_data_test = generate_missing_data(test_set, missing_rate)

        #保存生成的数据
        missing_data_train.to_csv(f'datasets/{DATANAME}\{DATANAME}_train_RANDOM_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)
        missing_data_val.to_csv(f'datasets/{DATANAME}\{DATANAME}_val_RANDOM_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)
        missing_data_test.to_csv(f'datasets/{DATANAME}\{DATANAME}_test_RANDOM_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)
        train_set.to_csv(f'datasets/{DATANAME}\{DATANAME}_train_REAL_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)# 选项2，保存到特定位置
        val_set.to_csv(f'datasets/{DATANAME}\{DATANAME}_val_REAL_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)# 选项2，保存到特定位置
        test_set.to_csv(f'datasets/{DATANAME}\{DATANAME}_test_REAL_{MISSING_RATE}%_NUM_{ID_NUM}.csv',header=None,index=False)# 选项2，保存到特定位置